In [1]:
import math
import logging
import numpy as np


logging.basicConfig()
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

class Activation(object):

    @staticmethod
    def relu_activation(x):
        return max(0, x)

    @staticmethod
    def sigmoid(x):
        return 1 / (1 + np.exp(-x))

    @staticmethod
    def tanh(x):
        return (1 - np.exp(x)) / (1 + np.exp(x))

    @staticmethod
    def softmax(x):
        x_new = [np.exp(i) for i in x]
        sum_x_new = sum(x_new)
        return [sum_x_new / (i) for i in x_new]

    @staticmethod
    def derivate_relu(x):
        if x > 0:
            return 1
        return 0

    @staticmethod
    def derivate_sigmoid(x):
        return (Activation.sigmoid(x)) * (1 - Activation.sigmoid(x))

    @staticmethod
    def derivate_tanh(x):
        return - np.exp(x) / (1 + np.exp(x)) ** 2


In [2]:
class LossFunction(object):

    @staticmethod
    def cross_entropy(Y_pred, Y_train):
        if Y_pred == 1:
            return -np.log(Y_train)
        else:
            return -np.log(1 - Y_train)

    @staticmethod
    def hinge_loss(Y_pred, Y_train):
        return np.max(0, 1 - Y_pred * Y_train)

    @staticmethod
    def L1_loss(Y_pred, Y_train):
        return np.sum(np.absolute(Y_pred - Y_train))

    @staticmethod
    def L2_loss(Y_pred, Y_train):
        return np.sum(np.power((Y_pred - Y_train), 2)) / len(Y_train)


In [3]:
class NeuralNetwork(object):

        def __init__(self, train_x, train_y, hidden_layer=1, hidden_neurons=3, bias = 1):
            self.train_x = train_x
            self.train_y = train_y
            
            self.hidden_layer = hidden_layer
            self.hidden_neurons = hidden_neurons
            self.input_nodes = np.shape(train_x)[0]
            self.output_nodes = np.shape(train_y)[0]
            self.bias = bias

            self.loop_counter = self.hidden_layer + 1
            # seed for the fixed random values
            np.random.seed(3)
            self.W_in = np.random.normal(0.0, 0.1, (self.input_nodes, self.hidden_neurons))
            self.W_out = np.random.normal(0.0, 0.1, (self.hidden_neurons, self.output_nodes))
            self.bias_weight_Mat = np.random.normal(0.0, 0.1, (1,self.loop_counter))
            
            # special case to check, override bias_weight_Mat
            self.bias_weight_Mat = [0.35, 0.65]          
            print(self.W_in, self.W_out, "First and last weight mat")
            self.Weight_Mat = list()
            self.Weight_Mat.append(self.W_in)
            
            if self.loop_counter > 2:
                for i in range(1, self.loop_counter-1):
                    self.Wh_i = np.random.normal(0.0, 0.1, (self.hidden_neurons, self.hidden_neurons))
                    self.Weight_Mat.append(self.Wh_i)
            self.Weight_Mat.append(self.W_out)
             
                    
        def L2_loss(self, Y_pred, Y_train):
            return np.sum(np.power((Y_pred - Y_train), 2)) / len(Y_train)
        
        def sigmoid(self, x):
            return 1 / (1 + np.exp(-x))
        
        def derivative_L2_loss(x):
            return -x

        def derivative_L1_loss(x):
            return -1

        def forward_prop(self):
            weight = self.Weight_Mat
            # Layewise activation output
            self.lw_net = list()
            self.lw_act_out = list()
            
            for i in range(len(weight)):
                print((weight[i]), i)
                print((self.train_x), i)
                X_i = np.dot(np.array(self.train_x).T, np.array(weight[i])) + self.bias_weight_Mat[i]
                self.lw_net.append(X_i.tolist()[0])
                A_i = self.sigmoid(X_i)
                self.lw_act_out.append(A_i.tolist()[0])
                self.train_x = (A_i).T
            self.E_total = 0
            
            for i in range(len(A_i)):
                E = self.L2_loss(np.array(A_i[i]), np.array(self.train_y[i]))
                self.E_total += E
            print("lw", self.lw_net)
            print("lw_ac_out", self.lw_act_out)
            print("total", self.E_total)
                        
        
#       backpropagation
        def back_prop(self):
            self.loss = self.E_total
            for i in reversed(range(self.loop_counter)):
                # output layer gradient
                self.grad_out = list()
                recur_grad = list()
                for k in range(len(self.lw_act_out[i])):
                            dloss_to_out = self.train_y[k][0] - self.lw_act_out[i][k]
                            dout_to_net = self.lw_act_out[i][k]*(1-self.lw_act_out[i][k])
                            recur_grad.append(dloss_to_out*dout_to_net)
                            print(recur_grad, "recur")
                            for p in range(len(self.lw_net[i])):
                                    dnet_to_weight = self.lw_net[i][p]
                                    self.grad_out.append(dloss_to_out*dout_to_net*dnet_to_weight)
                print(self.grad_out, "out")
            return self.grad_out
                
                
                
#
#             for el in self.lw_act_out[len(self.lw_act_out)-1]:
#                     for k in range(len(el)):
#                         loss_to_out = self.train_y[k][0] - el[k]
#                         out_to_net = el[k]*(1-el[k])
#                         for elm in self.lw_net[len(self.lw_net)-1]:
#                             for p in range(len(elm)):
#                                 net_to_weight = elm[p]
#                                 self.grad_out.append(loss_to_out*out_to_net*net_to_weight)
                                
#             for j in reversed(range(self.loop_counter-1)):
#                 for el in self.lw_act_out[j]:
#                     for k in range(len(el)):
#                         print(el[k])
# #                         loss_to_out = self.train_y[k][0] - el[k]
# #                         out_to_net = el[k]*(1-el[k])
# #                         for elm in self.lw_net[j]:
# #                             for p in range(len(elm)):
# #                                 net_to_weight = elm[p]
# #                                 self.grad_out.append(loss_to_out*out_to_net*net_to_weight)
           
            
        
        
        

In [4]:
nn = NeuralNetwork([[0.5], [0.1]], [[0.99], [0.01], [0.23]], 1, 3)
print(nn.forward_prop())
nn.back_prop()

[[ 0.17886285  0.04365099  0.00964975]
 [-0.18634927 -0.02773882 -0.0354759 ]] [[-0.00827415 -0.06270007 -0.00438182]
 [-0.0477218  -0.13138648  0.08846224]
 [ 0.0881318   0.17095731  0.00500336]] First and last weight mat
[[ 0.17886285  0.04365099  0.00964975]
 [-0.18634927 -0.02773882 -0.0354759 ]] 0
[[0.5], [0.1]] 0
[[-0.00827415 -0.06270007 -0.00438182]
 [-0.0477218  -0.13138648  0.08846224]
 [ 0.0881318   0.17095731  0.00500336]] 1
[[0.60367383]
 [0.59122979]
 [0.58692728]] 1
lw [[0.420796496637871, 0.36905161050045543, 0.35127728361091054], [0.6685175218763445, 0.6348095185571316, 0.7025929335798671]]
lw_ac_out [[0.603673828796172, 0.5912297942319985, 0.5869272825594465], [0.6611711281141477, 0.6535792046048169, 0.668762408001935]]
total 0.3245009690727652
None
[0.07366551561317096] recur
[0.07366551561317096, -0.14571497384806756] recur
[0.07366551561317096, -0.14571497384806756, -0.09719431939319241] recur
[0.049246687945460214, 0.04676357050065992, 0.05175687071833129, -0.0974

[0.038893886743043636,
 0.03411114792026228,
 0.0324682809703249,
 -0.05910926091886309,
 -0.05184065958698097,
 -0.049343900858793595,
 -0.03641352060219006,
 -0.03193578969787533,
 -0.030397692723319266]